# 4. Regime Detection (HMM)

Trains a 3-state HMM on first 70% and maps regimes to {-1,0,+1}.


In [ ]:

import pandas as pd
from pathlib import Path
from src.regime import fit_hmm

DATA_DIR = Path("../data")
df = pd.read_csv(DATA_DIR/"nifty_features_5min.csv", parse_dates=["timestamp"])

hmm_features = ["avg_iv","iv_spread","pcr_oi","call_delta","call_gamma","call_vega","futures_basis","spot_ret"]
for c in hmm_features:
    if c not in df.columns:
        df[c] = 0.0

df = df.dropna(subset=["spot_ret"]).reset_index(drop=True)

reg_raw, model = fit_hmm(df, hmm_features)
df = df.iloc[-len(reg_raw):].copy()
df["regime_raw"] = reg_raw

means = df.groupby("regime_raw")["spot_ret"].mean().sort_values()
down = means.index[0]
up   = means.index[-1]
mapping = {down:-1, up:1}
df["regime"] = df["regime_raw"].map(lambda x: mapping.get(x, 0))

out = pd.read_csv(DATA_DIR/"nifty_features_5min.csv", parse_dates=["timestamp"])
out = out.merge(df[["timestamp","regime"]], on="timestamp", how="left")
out.to_csv(DATA_DIR/"nifty_features_5min.csv", index=False)

print(out["regime"].value_counts(dropna=False))
